# Scrape website 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' for all Toronto neighborhoods

import libraries for data analysis (np and pd) and accessing website data (requests and BeautifulSoup)

In [2]:
import requests
from bs4 import BeautifulSoup
#import time
import numpy as np
import pandas as pd

access website and download all text

In [14]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
website_url = requests.get(url).text
soup = BeautifulSoup(website_url,'lxml')


Read in main table data to variable My_table with column headers PostCode, Borough, and Neighborhood.  Read by table row and separate by text td.  Drop missing values.  Strip out strings \n

In [15]:
My_table = soup.find('table',{'class':'wikitable sortable'})
rows  = My_table.find_all('tr')
TN=[]
for tr in rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    #print(row)
    TN.append(row)
df=pd.DataFrame(TN, columns=['Postal Code', 'Borough', 'Neighborhood'])
df = df.replace(to_replace='None', value=np.nan).dropna()
df['Neighborhood']=df['Neighborhood'].str.strip('\n')


Drop row if Borough Not assigined.  
If Neighborhood set to Not assigned replace with borough name.  
Group by postcode and borough.  
Assign one row for each postal code, in neighborhood column place all neighborhoods separated by comma.
Place Result into variable TN

In [36]:

df['Borough'] = df['Borough'].replace(to_replace="Not assigned", value=np.nan)
df=df.dropna()

df['Neighborhood'] = df['Neighborhood'].replace(to_replace="Not assigned", value=np.nan)
values=df['Borough']
df['Neighborhood'] = values.where(df['Neighborhood'].isnull(), other=df['Neighborhood'])

TN=df.groupby(['Postal Code', 'Borough'], sort=False)['Neighborhood'].apply(','.join).reset_index()

TN.columns=('Postal Code', 'Borough', 'Neighborhood')


Find number of rows of data with .shape

In [37]:
TN.shape

(103, 3)